# Part 1 
#### (keep scrolling for part 2)

In [1]:
%pip install lxml #installed lxml package to help parse html files
import numpy as np 
import pandas as pd
import lxml.html #parses html objects
import wikipedia as wp #installed to easily access Wikipedia content
from bs4 import BeautifulSoup #installed to pull data out of HTML and XML files
print('Libraries imported.')

Note: you may need to restart the kernel to use updated packages.
Libraries imported.


#### obtaining the content from wikipedia using the wp.page code to retreive the page data. the .html and .encode("UTF-8") used to change coding of languge to UTF-8

In [47]:
url=wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df=pd.read_html(url,header=0)[0]

#### if a borough is classified as 'not assigned', the row will be removed

In [48]:
df = df[df.Borough != 'Not assigned']

#### grouping the data by postal code and borough and joining rows together which may share neighborhoods. Finally reseting the index

In [49]:
df = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(list).apply(lambda x:', '.join(x)).to_frame().reset_index()

#### if neighborhood value is 'not assigned' the borough value will be assigned in place of the neighborhood value.

In [50]:
for index, row in df.iterrows():
    if row['Neighbourhood'] == 'Not assigned':
        row['Neighbourhood'] = row['Borough']

# Part 2
#### obtaining geospatial data to get the longitude and latitude of the different postal codes

In [51]:
import requests
import io
geo_url="http://cocl.us/Geospatial_data"
Geo_Data=requests.get(geo_url).content
Long_Lat=pd.read_csv(io.StringIO(Geo_Data.decode('utf-8')))
Long_Lat

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


#### Merging the two dataframes

In [52]:
df=pd.merge(Long_Lat,df,on='Postal Code')
df

,Postal Code,Latitude,Longitude,Borough,Neighbourhood
0,M1B,43.806686,-79.194353,Scarborough,"Malvern, Rouge"
1,M1C,43.784535,-79.160497,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,43.763573,-79.188711,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,43.770992,-79.216917,Scarborough,Woburn
4,M1H,43.773136,-79.239476,Scarborough,Cedarbrae
...,...,...,...,...,...
98,M9N,43.706876,-79.518188,York,Weston
99,M9P,43.696319,-79.532242,Etobicoke,Westmount
100,M9R,43.688905,-79.554724,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
101,M9V,43.739416,-79.588437,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


#### reordering the columns

In [54]:
df=df[['Postal Code','Borough','Neighbourhood','Latitude','Longitude']]
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ...",43.688905,-79.554724
101,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest...",43.739416,-79.588437
